In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

# Path where your LoRA-fine-tuned model was saved
peft_model_path = "./qwen_lora_finetuned/"

# Load LoRA config
peft_config = PeftConfig.from_pretrained(peft_model_path)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,  
    device_map="cpu"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load the LoRA weights
model = PeftModel.from_pretrained(base_model, peft_model_path)

model.eval()  # Put the model in evaluation mode


/home/zayn-husyn/Desktop/proj_prac/llm_feature/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


PeftModel(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=896, out_fea

In [2]:
prompt = (
    "You are an AI tutor. "
    "Explain Big O notation in simple terms, focusing on time complexity."
    " Provide an example and keep it under 100 words."
)

inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cpu")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=800,  # limit the number of new tokens
        do_sample=True,      # enable sampling
        top_k=40,            # top-k sampling
        top_p=0.9,           # nucleus sampling
        temperature=0.7,     # controls "creativity"
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


You are an AI tutor. Explain Big O notation in simple terms, focusing on time complexity. Provide an example and keep it under 100 words. Big O notation is a mathematical notation used to describe the upper bound of the growth rate of a function. It is used to describe the amount of time or space an algorithm will take to run as it increases in complexity. For example, if we have a function that takes O(n^2) time, it means that the function will take a quadratic amount of time to run as the input size increases by a factor of 2.
